In [0]:
from pyspark.sql import functions as F
from delta.tables import *

df_chamado = spark.read.table("v_credit.silver.tb_chamado")
df_log = spark.read.table("v_credit.silver.tb_chamado_log")
df_custo = spark.read.table("v_credit.silver.tb_custo_chamado")
df_pesquisa = spark.read.table("v_credit.silver.tb_pesquisa")

TABELA_DESTINO = "v_credit.gold.ft_atendimento_geral"

### Configuracao e Leitura de Tabelas Silver
Le 4 tabelas Silver:
- tb_chamado (dados principais)
- tb_chamado_log (timestamps)
- tb_custo_chamado (custos)
- tb_pesquisa (CSAT)

# Transformacao Silver → Gold: ft_atendimento_geral

## Proposito
Este notebook cria o **fato principal** do Data Warehouse dimensional, consolidando dados de multiplas tabelas Silver em uma **Fact Table otimizada para analytics**.

## O que e uma Fact Table?
Uma tabela fato em um Star Schema contem:
- **Foreign Keys** para dimensoes (clientes, canais, atendentes, motivos)
- **Metricas** (tempos, custos, notas)
- **Flags** para filtros (resolvido, experiencia negativa)
- **Degenerate Dimensions** (cd_chamado - ID transacional)

## Arquitetura Star Schema:

```
dm_atendente
     |
     |
dm_cliente ← ft_atendimento_geral → dm_canal
     |              |
     |              |
dm_motivo      dm_chamado
```

## O que este notebook faz:

### 1. Joins Multiplos
Combina 4 tabelas Silver:
- `tb_chamado`: Dados principais do ticket
- `tb_chamado_log`: Timestamps detalhados
- `tb_custo_chamado`: Custos associados
- `tb_pesquisa`: Notas de satisfacao (CSAT)

### 2. Criacao de Surrogate Key
- **pk_fato_atendimento**: Hash MD5 de natural keys
- Garante unicidade mesmo com dados distribuidos
- Facilita MERGE (upsert)

### 3. Derivacao de Metricas
- **dt_referencia**: Extrai DATE do timestamp
- **nu_hora_dia**: Extrai hora (0-23) para analise temporal
- **val_tempo_espera / val_tempo_atendimento**: Converte para formato analitico
- **fl_experiencia_negativa**: Flag calculada

### 4. Regra de Negocio Critica: Experiencia Negativa
```sql
fl_experiencia_negativa = 1 QUANDO:
  - Tempo de Espera > 300 segundos (5 minutos) OU
  - Nota CSAT <= 2
```

Esta flag responde a dor da **Diretora de Marketing**: "48% dos clientes estao tendo experiencia negativa"

### 5. Tratamento de NULLs
- Usa `F.coalesce()` para substituir NULL por 0
- Garante que metricas sejam sempre calculaveis
- Evita erros em agregacoes

## Decisoes Tecnicas:

### Por que Surrogate Key (Hash)?
- **Distribuicao**: Garante distribuicao uniforme em Spark
- **Idempotencia**: Mesmo input → mesma key
- **Sem Conflitos**: Evita problemas de auto-increment distribuido

### Por que LEFT JOIN?
- Nem todo chamado tem custo associado
- Nem todo chamado tem pesquisa de satisfacao
- LEFT JOIN garante que nao perdemos registros

### Por que MERGE?
- Permite reprocessamento sem duplicatas
- Suporta cargas incrementais futuras
- Aproveita ACID do Delta Lake

## Metricas de Negocio Respondidas:

| Stakeholder | Metrica | Campo |
|-------------|---------|-------|
| Gerente Financeiro | Custo por atendimento | val_custo |
| Diretora Marketing | CSAT e Exp. Negativa | val_nota_csat, fl_experiencia_negativa |
| Coordenadora CS | Tempos de espera | val_tempo_espera |
| Diretor TI | Transbordo Bot → Humano | cd_atendente (NULL = Bot) |

## Dependencias:
- Silver carregado (tb_chamado, tb_chamado_log, tb_custo_chamado, tb_pesquisa)
- DDL Gold executado (`ddl_ft_atendimento_geral.ipynb`)

## Proximo passo:
Criar views Curated para consumo no Power BI.

### Joins e Criacao do Fato Principal
Faz LEFT JOINs para consolidar dados e cria tabela fato:
- **pk_fato_atendimento**: Surrogate key (MD5 hash)
- **FKs**: cd_cliente, cd_motivo, cd_canal, cd_atendente
- **Metricas**: val_tempo_espera, val_tempo_atendimento, val_custo, val_nota_csat
- **Flags de Negocio**:
  - st_resolvido: 1 se resolvido
  - **fl_experiencia_negativa**: 1 quando (Espera > 5min OU Nota <= 2)

Faz MERGE em ft_atendimento_geral (Gold).

**Importante**: Fato central do Star Schema usado para responder todas as dores de negocio dos stakeholders.

In [0]:
df_join = (
    df_chamado.alias("c")
    .join(df_log.alias("l"), on="cd_chamado", how="left")
    .join(df_custo.alias("cust"), on="cd_chamado", how="left")
    .join(df_pesquisa.alias("p"), on="cd_chamado", how="left")
)

df_gold = df_join.select(
    F.md5(F.concat(
        F.coalesce(F.col("c.cd_cliente").cast("string"), F.lit("0")),
        F.coalesce(F.col("c.cd_motivo").cast("string"), F.lit("0")),
        F.coalesce(F.col("c.cd_canal").cast("string"), F.lit("0")),
        F.coalesce(F.col("c.cd_atendente").cast("string"), F.lit("0")),
        F.coalesce(F.col("c.cd_chamado").cast("string"), F.lit("0"))
    )).alias("pk_fato_atendimento"),
    
    F.col("c.cd_cliente").cast("string"),
    F.col("c.cd_motivo"),
    F.col("c.cd_canal"),
    F.col("c.cd_atendente"),
    
    F.col("c.cd_chamado").cast("bigint").alias("cd_chamado"),
    
    F.to_date(F.coalesce(F.col("l.dh_inicio"), F.col("c.dt_ingestion"))).alias("dt_referencia"),
    F.hour(F.coalesce(F.col("l.dh_inicio"), F.col("c.dt_ingestion"))).cast("smallint").alias("nu_hora_dia"),
    
    F.coalesce(F.col("c.tm_espera"), F.lit(0)).cast("bigint").alias("val_tempo_espera"),
    F.coalesce(F.col("c.tm_duracao"), F.lit(0)).cast("bigint").alias("val_tempo_atendimento"),
    
    F.coalesce(F.col("cust.vl_custo"), F.lit(0)).cast("decimal(12,10)").alias("val_custo"),
    
    F.col("p.nu_nota").cast("smallint").alias("val_nota_csat"),
    
    F.when(F.col("c.st_resolvido") == True, 1).otherwise(0).cast("smallint").alias("st_resolvido"),
    
    F.when(
        (F.col("c.tm_espera") > 300) | (F.col("p.nu_nota") <= 2), 1
    ).otherwise(0).cast("smallint").alias("fl_experiencia_negativa")
)



DeltaTable.forName(spark, TABELA_DESTINO).alias("t").merge(
    df_gold.alias("s"), "t.pk_fato_atendimento = s.pk_fato_atendimento"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

print(f"✅ Fato Atendimento Geral carregada (Linhagem preservada)!")